In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc
import sys
sys.path.insert(0, '../../Code')
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv("../../datasets/titanicPreprocesado.csv")

In [3]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamSize',
       'Title'],
      dtype='object')

In [4]:
df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,FamSize,Title
0,0,3,male,22.000000,7.2500,S,2,Mr
1,1,3,female,26.000000,7.9250,S,1,Miss
2,1,1,female,35.000000,53.1000,S,2,Mrs
3,0,3,male,35.000000,8.0500,S,1,Mr
4,0,3,male,29.699118,8.4583,Q,1,Mr


In [5]:

#Fare => Tarifa => binning
#df['Fare'] = pd.qcut(df['Fare'], q=4, labels=False)
df["binFare"] = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile').fit_transform(df[["Fare"]]).astype(int)
df["binAge"] = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile').fit_transform(df[["Fare"]]).astype(int)
dfNB = df.drop(columns=["Fare", "Age"])


In [6]:
dfNB.head()
#[Sex, Embarked, Title]

,Survived,Pclass,Sex,Embarked,FamSize,Title,binFare,binAge
0,0,3,male,S,2,Mr,0,0
1,1,3,female,S,1,Miss,1,1
2,1,1,female,S,2,Mrs,3,4
3,0,3,male,S,1,Mr,1,2
4,0,3,male,Q,1,Mr,1,2


In [7]:
dfNB['Title'].unique()

array(['Mr', 'Miss', 'Mrs', 'No Conocido'], dtype=object)

El ordinal tiene mas sentido en este caso que el OneHotEncoder

In [8]:
oe = OrdinalEncoder(categories=[['male', 'female'], ['C', 'Q', 'S'], ['Mr', 'Mrs', 'Miss', 'No Conocido']])
oe.fit(dfNB[['Sex','Embarked', 'Title']])
dfNB.loc[:,['Sex','Embarked', 'Title']] = oe.transform(dfNB[['Sex','Embarked', 'Title']]).astype(int)

In [9]:
dfNB.head()

,Survived,Pclass,Sex,Embarked,FamSize,Title,binFare,binAge
0,0,3,0,2,2,0,0,0
1,1,3,1,2,1,2,1,1
2,1,1,1,2,2,1,3,4
3,0,3,0,2,1,0,1,2
4,0,3,0,1,1,0,1,2


In [10]:
X_train, X_test, y_train, y_test = train_test_split(dfNB.drop(columns="Survived"), dfNB["Survived"], test_size=0.2, random_state=0)

In [11]:
model = MultinomialNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7803030303030303